# Cohort 4 Screening Task 1

Following is one possible solution to the Task 1. We begin by importing the appropriate libraries:

In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

Next, we will create our circuit and define our oracle function. The idea is to have an equal superposition of 8-qubit states to begin with, and to use Grover's algorithm to keep only those that obey the target condition (where the first four digits would correspond to one number in binary form and the last four to another). For that, we create a clause list in which we list the indeces of the digits that need to be compared. 

In [6]:
clause_list = [[0, 4], [1, 5], [2, 6], [3, 7]]

Next, we think about how we will compare the necessary indeces. We want them to be opposite, so we are looking for a xor operation. We might define one as:

In [2]:
def XOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)

Now, we declare our quantum circuit. We want a superposition of 8-qubit states, which we store in the var_qubits quantum register. To store the result of checking each clause, we create a clause_qubits register as well. We will need an output qubit which will have its value depend on the values of the clause qubits (if all clauses are satisfied, the output will be 1). Finally, we want 8 classical bits to measure the var_qubits.

In [4]:
var_qubits = QuantumRegister(8, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(8, name='cbits')

qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

Our oracle should simply check all clauses, switch the value of the corresponding clause qubit when a clause is obeyed, and flip the output qubit when all clauses are obeyed. Additionally, in order to use Grover's algorithm later, we need to 'uncompute' the clause qubits once we have decided whether or not the output should be flipped. That way, we can repeat the process, and we have created an oracle:

In [7]:
def oracle(qc, clause_list, clause_qubits):
    i=0
    for clause in clause_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i+=1
        
    qc.mct(clause_qubits, output_qubit)
    
    i=0
    for clause in clause_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i+=1

oracle(qc, clause_list, clause_qubits)   
qc.draw()

»
    v_0: ──■────────────────────────────────────────────■──────────────────────»
           │                                            │                      »
    v_1: ──┼────■───────────────────────────────────────┼────■─────────────────»
           │    │                                       │    │                 »
    v_2: ──┼────┼────■──────────────────────────────────┼────┼────■────────────»
           │    │    │                                  │    │    │            »
    v_3: ──┼────┼────┼────■─────────────────────────────┼────┼────┼────■───────»
           │    │    │    │                             │    │    │    │       »
    v_4: ──┼────┼────┼────┼────■────────────────────────┼────┼────┼────┼────■──»
           │    │    │    │    │                        │    │    │    │    │  »
    v_5: ──┼────┼────┼────┼────┼────■───────────────────┼────┼────┼────┼────┼──»
           │    │    │    │    │    │                   │    │    │    │    │  »
    v_6: ──┼────┼────┼────┼────┼────┼────■──────────────┼────┼────┼────┼────┼──»
           │    │    │    │    │    │    │              │    │    │    │    │  »
    v_7: ──┼────┼────┼────┼────┼────┼────┼────■─────────┼────┼────┼────┼────┼──»
         ┌─┴─┐  │    │    │  ┌─┴─┐  │    │    │       ┌─┴─┐  │    │    │  ┌─┴─┐»
    c_0: ┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼────■──┤ X ├──┼────┼────┼──┤ X ├»
         └───┘┌─┴─┐  │    │  └───┘┌─┴─┐  │    │    │  └───┘┌─┴─┐  │    │  └───┘»
    c_1: ─────┤ X ├──┼────┼───────┤ X ├──┼────┼────■───────┤ X ├──┼────┼───────»
              └───┘┌─┴─┐  │       └───┘┌─┴─┐  │    │       └───┘┌─┴─┐  │       »
    c_2: ──────────┤ X ├──┼────────────┤ X ├──┼────■────────────┤ X ├──┼───────»
                   └───┘┌─┴─┐          └───┘┌─┴─┐  │            └───┘┌─┴─┐     »
    c_3: ───────────────┤ X ├───────────────┤ X ├──■─────────────────┤ X ├─────»
                        └───┘               └───┘┌─┴─┐               └───┘     »
  out_0: ────────────────────────────────────────┤ X ├─────────────────────────»
                                                 └───┘                         »
cbits: 8/══════════════════════════════════════════════════════════════════════»
                                                                               »
«                        
«    v_0: ───────────────
«                        
«    v_1: ───────────────
«                        
«    v_2: ───────────────
«                        
«    v_3: ───────────────
«                        
«    v_4: ───────────────
«                        
«    v_5: ──■────────────
«           │            
«    v_6: ──┼────■───────
«           │    │       
«    v_7: ──┼────┼────■──
«           │    │    │  
«    c_0: ──┼────┼────┼──
«         ┌─┴─┐  │    │  
«    c_1: ┤ X ├──┼────┼──
«         └───┘┌─┴─┐  │  
«    c_2: ─────┤ X ├──┼──
«              └───┘┌─┴─┐
«    c_3: ──────────┤ X ├
«                   └───┘
«  out_0: ───────────────
«                        
«cbits: 8/═══════════════
«

Finally, we simply put the oracle into Grover's algorithm, remembering to define a general diffuser first (as in the Qiskit textbook, chapter 3.8)

In [8]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

var_qubits = QuantumRegister(8, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(8, name='cbits')
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

# Initialize 'out0' in state |->
qc.initialize([1, -1]/np.sqrt(2), output_qubit)

# Initialize qubits in state |s>
qc.h(var_qubits)
qc.barrier()  # for visual separation

## First Iteration
# Apply our oracle
oracle(qc, clause_list, clause_qubits)
qc.barrier()  # for visual separation
# Apply our diffuser
qc.append(diffuser(4), [0,1,2,3])

## Second Iteration
oracle(qc, clause_list, clause_qubits)
qc.barrier()  # for visual separation
# Apply our diffuser
qc.append(diffuser(4), [0,1,2,3])

# Measure the variable qubits
qc.measure(var_qubits, cbits)

qc.draw(fold=-1)

┌───┐               ░                                                                                       ░ ┌────────┐                                                                                      ░ ┌────────┐         ┌─┐         
    v_0: ─────────────┤ H ├───────────────░───■────────────────────────────────────────────■──────────────────────────────────────░─┤0       ├──■────────────────────────────────────────────■──────────────────────────────────────░─┤0       ├─────────┤M├─────────
                      ├───┤               ░   │                                            │                                      ░ │        │  │                                            │                                      ░ │        │         └╥┘┌─┐      
    v_1: ─────────────┤ H ├───────────────░───┼────■───────────────────────────────────────┼────■─────────────────────────────────░─┤1       ├──┼────■───────────────────────────────────────┼────■─────────────────────────────────░─┤1       ├──────────╫─┤M├──────
                      ├───┤               ░   │    │                                       │    │                                 ░ │  U$_s$ │  │    │                                       │    │                                 ░ │  U$_s$ │          ║ └╥┘┌─┐   
    v_2: ─────────────┤ H ├───────────────░───┼────┼────■──────────────────────────────────┼────┼────■────────────────────────────░─┤2       ├──┼────┼────■──────────────────────────────────┼────┼────■────────────────────────────░─┤2       ├──────────╫──╫─┤M├───
                      ├───┤               ░   │    │    │                                  │    │    │                            ░ │        │  │    │    │                                  │    │    │                            ░ │        │          ║  ║ └╥┘┌─┐
    v_3: ─────────────┤ H ├───────────────░───┼────┼────┼────■─────────────────────────────┼────┼────┼────■───────────────────────░─┤3       ├──┼────┼────┼────■─────────────────────────────┼────┼────┼────■───────────────────────░─┤3       ├──────────╫──╫──╫─┤M├
                      ├───┤               ░   │    │    │    │                             │    │    │    │                       ░ └────────┘  │    │    │    │                             │    │    │    │                       ░ └──┬─┬───┘          ║  ║  ║ └╥┘
    v_4: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────■────────────────────────┼────┼────┼────┼────■──────────────────░─────────────┼────┼────┼────┼────■────────────────────────┼────┼────┼────┼────■──────────────────░────┤M├──────────────╫──╫──╫──╫─
                      ├───┤               ░   │    │    │    │    │                        │    │    │    │    │                  ░             │    │    │    │    │                        │    │    │    │    │                  ░    └╥┘    ┌─┐       ║  ║  ║  ║ 
    v_5: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────■───────────────────┼────┼────┼────┼────┼────■─────────────░─────────────┼────┼────┼────┼────┼────■───────────────────┼────┼────┼────┼────┼────■─────────────░─────╫─────┤M├───────╫──╫──╫──╫─
                      ├───┤               ░   │    │    │    │    │    │                   │    │    │    │    │    │             ░             │    │    │    │    │    │                   │    │    │    │    │    │             ░     ║     └╥┘┌─┐    ║  ║  ║  ║ 
    v_6: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────┼────■──────────────┼────┼────┼────┼────┼────┼────■────────░─────────────┼────┼────┼────┼────┼────┼────■──────────────┼────┼────┼────┼────┼────┼────■────────░─────╫──────╫─┤M├────╫──╫──╫──╫─
                      ├───┤               ░   │    │    │    │    │    │    │              │    │    │    │    │    │    │        ░             │    │    │    │    │    │    │              │    │    │    │    │    │    │        ░     ║      ║ └╥┘┌─┐ ║  ║  ║  ║ 
    v_7: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────┼────┼────■─────────┼

From this circuit, we can obtain the possible 8-digit combinations that would obey the condition required. Below, we pick a backend simulator and obtain the results.

In [9]:
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend)
qobj = assemble(transpiled_qc)
result = backend.run(qobj).result()

Now, because of error, not all combinations that got a count will actually be solutions to the problem. We can see the number of counts for each option as follows:

In [10]:
options = result.get_counts()
print(options)

{'00000001': 1, '00010110': 1, '00011011': 2, '00011110': 53, '00000010': 1, '00100010': 1, '00100110': 1, '00101101': 54, '00000011': 1, '00110101': 1, '00110111': 1, '00111001': 1, '00111100': 58, '00111101': 1, '00111110': 1, '00000100': 1, '01000010': 1, '01000111': 1, '01001001': 1, '01001011': 71, '01001100': 1, '01001101': 1, '01001111': 1, '01010011': 1, '01010100': 1, '01010111': 1, '01011010': 67, '01011110': 2, '01100101': 1, '01100110': 1, '01100111': 1, '01101001': 65, '01101101': 1, '01110101': 1, '01111000': 62, '01111001': 1, '01111010': 1, '01111101': 1, '10000011': 1, '10000101': 1, '10000110': 1, '10000111': 50, '10001101': 2, '10001111': 1, '10010100': 2, '10010101': 2, '10010110': 66, '10011001': 1, '10011010': 1, '10011110': 1, '10100101': 52, '10100110': 1, '10101000': 1, '10101001': 1, '10101010': 1, '10101111': 1, '10110000': 1, '10110001': 1, '10110100': 56, '10110101': 1, '10111001': 1, '10111011': 1, '10111111': 1, '00001100': 1, '11000011': 59, '11000100': 

Clearly, the options that actually are answers to the problem got 40 or more counts. Therefore, we can create a list of possible answers:

In [11]:
answers = []
for (opt, val) in options.items():
    if val>40:
        answers.append(opt)

In [12]:
print(answers)

['00011110', '00101101', '00111100', '01001011', '01011010', '01101001', '01111000', '10000111', '10010110', '10100101', '10110100', '11000011', '11010010', '11100001', '00001111', '11110000']


All of this has given us the possible combinations that would make an input a solution: if a given input had the first four digits and the last four digits of one of these options as elements, it would have two numbers that satisfy the target condition. But in order for us to be able to given an input and have the superposition of the indeces of such numbers returned, we need a new function and a new circuit for our final superposition state. Those are defined below. The idea behind the solution function is to find all of the 8-digit combinations of the binary forms of the numbers in the input, and to check if they are included among the answers. Then, depending on what the indeces of the numbers that satisfy this are, a different superposition state is returned.

In [13]:
def decimalToBinary(n):
    return bin(n).replace("0b", "")
  
final_state = QuantumRegister(2, name='final')
qc = QuantumCircuit(final_state)

def solution(inp):
    biInp=[]
    i=0
    while i<4:
        biInp.append(decimalToBinary(inp[i]).zfill(4))
        i+=1
    
    combs={}
    j=0
    while j<3:
        combs[str(j)+str(j+1)]=biInp[j]+biInp[j+1]
        j+=1
    j=0
    while j<2:
        combs[str(j)+str(j+2)]=biInp[j]+biInp[j+2]
        j+=1
    combs['03']=biInp[0]+biInp[3]
    
    for (k, v) in combs.items():
        for ans in answers:
            if v==ans:
                k1=int(k[0])
                k2=int(k[1])
                kb1=decimalToBinary(int(k[0])).zfill(2)
                kb2=decimalToBinary(int(k[1])).zfill(2)
                if k1==0 and k2==1:
                    qc.initialize([1, 1, 0, 0]/np.sqrt(2), final_state)
                elif k1==0 and k2==2:
                    qc.initialize([1, 0, 1, 0]/np.sqrt(2), final_state)
                elif k1==0 and k2==3:
                    qc.initialize([1, 0, 0, 1]/np.sqrt(2), final_state)
                elif k1==1 and k2==2:
                    qc.initialize([0, 1, 1, 0]/np.sqrt(2), final_state)
                elif k1==1 and k2==3:
                    qc.initialize([0, 1, 0, 1]/np.sqrt(2), final_state)
                else:
                    qc.initialize([0, 0, 1, 1]/np.sqrt(2), final_state)

    # Select the StatevectorSimulator from the Aer provider
    simulator = Aer.get_backend('statevector_simulator')

    # Execute and get counts
    result = execute(qc, simulator).result()
    statevector = result.get_statevector(qc)
    print(statevector)

    
solution([1, 5, 7, 10])

solution([1, 5, 4, 2])

[0.        +0.j 0.70710678+0.j 0.        +0.j 0.70710678+0.j]
[0.        +0.j 0.70710678+0.j 0.        +0.j 0.70710678+0.j]
